## Get all the necessary data in order to visualize

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
# now just read the html to get all the S&P500 tickers 
dataload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = dataload[0]
# now get the first column(tickers) from the above data
# convert it into a list
ticker_list = df['Symbol'][25:35].values.tolist()
all_tickers = " ".join(ticker_list)
# get all the tickers from yfinance
tickers = yf.Tickers(all_tickers)
# set a start and end date to get two-years info
# group by the ticker
hist = tickers.history(start='2020-05-01', end='2022-05-01', group_by='ticker')
stock_data = pd.DataFrame(hist.stack(level=0).reset_index().rename(columns = {'level_1':'Ticker'}))
stock_data['DailyReturn'] = stock_data.sort_values(['Ticker', 'Date']).groupby('Ticker')['Close'].pct_change()
# add the column for moving average daily return, set the moving days to 100 days
stock_data['MovingAverageDailyReturn'] = stock_data.sort_values(['Ticker', 'Date']).groupby('Ticker')['DailyReturn'].rolling(100).mean().reset_index(0,drop=True) 
# Add a column containing the 100days moving average volume
stock_data['MovingAverageVolume'] = stock_data.sort_values(['Ticker', 'Date']).groupby('Ticker')['Volume'].rolling(100).mean().reset_index(0,drop=True)
# create a new Year column, to be the year period of each ticker
stock_data['Year'] = stock_data.Date.dt.to_period("Y")

[*********************100%***********************]  10 of 10 completed


In [3]:
# Calculate the volume-weighted daily returns
stock_data['VWDR'] = np.nan  # 

# function that calculates the volume weighted daily returns
# input should be the dataframe
def func_data(x):
    x['test'] = x['Volume'] * x['DailyReturn'] / x['Volume'].cumsum()
    return x
# create a new column of volume-weighted daily returns, groupby ticker, and apply the above function
stock_data['VWDR'] = stock_data.groupby(['Ticker']).apply(func_data).iloc[:, -1]

In [4]:
# create a reference data table, containing some useful information of 10 tickers
reference_table = pd.DataFrame.from_dict({k: [yf.Ticker(ticker).info[k] for ticker in ticker_list] \
                                           for k in ['symbol', 'longName']})
reference = reference_table.rename(columns={'symbol': 'Ticker', 'longName': 'Company'})

# create a transaction table; The transactions span over 2-year period
transactions = {'Date':['2020-05-26', '2020-05-27', '2020-07-30', '2021-01-18', '2021-03-17', '2021-05-21', '2021-09-30', '2021-11-22', '2021-12-06'
                       , '2022-01-06', '2022-02-14'], 
                'Ticker': ['AMZN', 'GOOG', 'AMD', 'AMT', 'AMCR', 'AXP', 'GOOG', 'MO', 'AEP', 'AAL', 'AMZN'],
               'Amount': [+2000, +2000, +1000, -2000, +1000, +2000, -1000, +1000, +1000, +2000, +1000],
                'Buy/Sell': ['Buy', 'Buy', 'Buy', 'Sell', 'Buy', 'Buy', 'Sell', 'Buy', 'Buy', 'Buy', 'Buy']
               }
transactions_table = pd.DataFrame(transactions)
transactions_table['Date'] = pd.to_datetime(transactions_table['Date'])  
transactions_table['Position'] = transactions_table.groupby('Ticker')['Amount'].cumsum()

In [5]:
def create_join_table(date):
    # merge the transaction table with the reference table on Ticker, this is to get the company's name for each ticker
    with_company_name = pd.merge(transactions_table[['Date', 'Ticker', 'Position']], reference[['Ticker', 'Company']], on=['Ticker'])
    # now group by the Date and Ticker with the merge list, and filter through the Date less than the date passed-in
    transaction_with_date = with_company_name.groupby(['Date', 'Ticker']).filter(lambda x: x.Date <= date)
    # This is to make sure that the function will drop the ticker with Position <=0 (in case we sell the stock later)
    transaction_to_drop = transaction_with_date[(transaction_with_date['Position'] <= 0)]
    drop_list = transaction_to_drop['Ticker'].tolist()
    if transaction_to_drop.shape[0] != 0:  # if we really have something in the transaction_to_drop
        #drop the transations with negative position values
        transaction_with_date.drop(transaction_with_date[(transaction_with_date['Position'] <= 0)].index, inplace=True)
        #drop all other occurrences of the transactions to drop
        for item in drop_list:
            transaction_with_date.drop(transaction_with_date[(transaction_with_date['Ticker'] == item)].index, inplace=True)
    # drop the first ticker, keep the last one (make it unique)
    transaction_with_date.drop_duplicates(subset='Ticker', keep='last', inplace=True, ignore_index=False)
    # pass the date entered to the transaction filter table
    transaction_with_date['Date']= date
    # convert the date to datetime (make sure it's datetime rather than object)
    transaction_with_date['Date'] = pd.to_datetime(transaction_with_date['Date'])
    # filter the stock historical data table to get the info on the date passed-in (do a filter)
    stock_with_date = stock_data.groupby(['Date', 'Ticker']).filter(lambda x: x.Date == date)
    # finally join the two list on inner
    join_list = transaction_with_date.merge(stock_with_date[['Date','Ticker', 'Open', 'Volume', 'DailyReturn']], on = ['Date','Ticker'], how='inner')
    return join_list

In [6]:
# now create a ‘scaffold’ table, which is the concatenation of the above function
# for every business date over the two-year period.
import numpy as np
res = []  # initialize an empty result list to append the result later
date_list = stock_data.Date.tolist()  # store the date from historical data table over two years to the date_list
x = np.array(date_list)  # make the list an array
unique_date_list = np.unique(x)  # we only include the unique date here
for date in unique_date_list:  # using the for loop to append each dataframe result to the list
    res.append(create_join_table(date))
scaffold_table = pd.concat(res, ignore_index=True)  # concate the result list finally
scaffold_table['Daily PnL'] = scaffold_table['DailyReturn'] * scaffold_table['Position']
scaffold_table['Cumulative PnL/Ticker'] = scaffold_table.groupby(['Ticker'])['Daily PnL'].cumsum()

# ipywidgets Visualization

In [7]:
from IPython.display import display
import ipywidgets as widgets
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

### Exercise 3 Part

#### Overlay the 2 return (VWDR, MovingAverage) on the same plot
#### Ticker List Overview

In [8]:
def DailyReturnBrowser(df):
    # plot both daily returns, so make subplots
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # a search engine for the tickers
    search = widgets.Text(value = '', placeholder='Search string', description='', disabled=False, continuous_update=True)
    
    # create the dropdown box for the ticker selections
    ticker = widgets.Dropdown(
        options=sorted(list(df['Ticker'].unique())),
        value=sorted(df['Ticker'].unique())[0],
        description='Ticker: ',
    )
    
    # display the use_year ticker search box and ticker dropdown box
    container = widgets.HBox(children=[search, ticker])

    # Assign an emptry figure widget with two traces
    g = go.FigureWidget(fig, layout=go.Layout(
                            width=1000, height=500,
                            title=dict(
                                text='Ticker Daily Return Overview'
                            ),
                        ))
    
    # create the line chart
    g.add_scatter(name='Volume Weighted Daily Return')
    g.add_scatter(name='Moving Average Daily Return', secondary_y=True)
        
    def response(change):
            temp_df = df[(df['Ticker'] == ticker.value)]
            with g.batch_update():
                # the primary y-axis will be Volume Weighted Daily return
                g.data[0].x = temp_df['Date'].tolist()
                g.data[0].y = temp_df['VWDR'].tolist()
                # the secondary y-axis will be Moving Average Daily Return (100 days)
                g.data[1].x = temp_df['Date'].tolist()
                g.data[1].y = temp_df['MovingAverageDailyReturn'].tolist()
                g.layout.xaxis.title = 'Date'
    
    def chTicker(change):
        if search.value:
            ticker.options = sorted(df[df['Ticker'].str.contains(search.value)]['Ticker'].unique())
        else:
            ticker.options = sorted(list(df['Ticker'].unique()))
    # these are to update each choice in the box selected by user        
    ticker.observe(response, names='value')
    search.observe(chTicker, names='value')
                       
    response(None)
    chTicker(None)
    # to display the boxes
    display(container)
    display(g)

In [9]:
DailyReturnBrowser(stock_data)

FigureWidget({
    'data': [{'name': 'Volume Weighted Daily Return',
              'type': 'scatter',
        …

#### Moving Average Volume

In [10]:
def MovingVolumeBrowser(df):

    search = widgets.Text(value = '', placeholder='Search string', description='', disabled=False, continuous_update=True)
        
    ticker = widgets.Dropdown(
        options=sorted(list(df['Ticker'].unique())),
        value=sorted(df['Ticker'].unique())[0],
        description='Ticker: ',
    )
    
    # display the use_year ticker search box and ticker dropdown box
    container = widgets.HBox(children=[search, ticker])

    # Assign an emptry figure widget with two traces
    g = go.FigureWidget(layout=go.Layout(
                            width=1000, height=500,
                            title=dict(
                                text='Ticker Moving Average Volume (100 days)'
                            ),
                        ))
    # create the line chart
    g.add_scatter(name='Moving Average Volume')
        
    def response(change):
            temp_df = df[(df['Ticker'] == ticker.value)]
            with g.batch_update():
                # x-axis to display the Date and y-axis to display the Moving Average Volume
                g.data[0].x = temp_df['Date'].tolist()
                g.data[0].y = temp_df['MovingAverageVolume'].tolist()
                g.layout.xaxis.title = 'Date'
                g.layout.yaxis.title = 'Moving Average Volume'
                
    
    def chTicker(change):
        if search.value:
            ticker.options = sorted(df[df['Ticker'].str.contains(search.value)]['Ticker'].unique())
        else:
            ticker.options = sorted(list(df['Ticker'].unique()))
    # these are to update each choice in the box selected by user        
    ticker.observe(response, names='value')
    search.observe(chTicker, names='value')
                       
    response(None)
    chTicker(None)
    # to display the boxes
    display(container)
    display(g)

In [11]:
MovingVolumeBrowser(stock_data)

FigureWidget({
    'data': [{'name': 'Moving Average Volume',
              'type': 'scatter',
              '…

### Exercise 4 Part

#### The Two-year Distribution Plot of Daily Volume

In [12]:
# instead of date, year would be selected in order to see the plot
def VolumeBrowser(df):
    # create a year slider to choose the year in order to display the Volume distribution
    year = widgets.IntSlider(
            description='Pick a Year',
            value = 2020,  # make value to be the first year in the stock data
            min = 2020,
            max = 2022,
            step=1.0,
            disabled=False
        )
    use_year = widgets.Checkbox(
        description='Use Year: ',
        value=True,
    )
    # display the use_year check box and the year slider
    container = widgets.HBox(children=[use_year, year])

    search = widgets.Text(value = '', placeholder='Search string', description='', disabled=False, continuous_update=True)
        
    ticker = widgets.Dropdown(
        options=sorted(list(df['Ticker'].unique())),
        value=sorted(df['Ticker'].unique())[0],
        description='Ticker: ',
    )
    
    container2 = widgets.HBox(children=[search, ticker])
    
    # create a histogram for the distribution
    trace1 = go.Histogram(x=df['Volume'], name='Arrival Delays')
    
    # Assign an emptry figure widget with two traces
    g = go.FigureWidget(data=[trace1], layout=go.Layout(
                            width=1000, height=500,
                            title=dict(
                                text='Distribution of Daily Volume'
                            ),
                        ))
    
    g.add_scatter(name='Daily Volume')
    def response(change):
        if use_year.value:
            year.disabled = False
            # if select year, make the volume displayed based on ticker and year
            temp_df = df[(df['Ticker'] == ticker.value) & (df['Year'] == str(year.value))]
            with g.batch_update():
                g.data[0].x = temp_df['Volume'].tolist()
                g.layout.xaxis.title = 'Volume'
                g.layout.yaxis.title = 'Count'

        else:
            year.disabled = True
            temp_df = df[(df['Ticker'] == ticker.value)]
            with g.batch_update():
                g.data[0].x = temp_df['Volume'].tolist()    
                g.layout.xaxis.title = 'Volume'
                g.layout.yaxis.title = 'Count'
    
    def chTicker(change):
        if search.value:
            ticker.options = sorted(df[df['Ticker'].str.contains(search.value)]['Ticker'].unique())
        else:
            ticker.options = sorted(list(df['Ticker'].unique()))
    
    # these are to update each choice in the box selected by user
    year.observe(response, names='value')
    use_year.observe(response, names='value')
    ticker.observe(response, names='value')
    search.observe(chTicker, names='value')
    # to display the boxes
    display(container)
    display(container2)
    display(g)

In [13]:
VolumeBrowser(stock_data)

FigureWidget({
    'data': [{'name': 'Arrival Delays',
              'type': 'histogram',
              'uid':…

### Exercise 5 Part

#### cumulative PnL for each ticker from my portfolio

In [14]:
def PnLBrowser(df):

    search = widgets.Text(value = '', placeholder='Search string', description='', disabled=False, continuous_update=True)
        
    ticker = widgets.Dropdown(
        options=sorted(list(df['Ticker'].unique())),
        value=sorted(df['Ticker'].unique())[0],
        description='Ticker: ',
    )
    
    # display the use_year ticker search box and ticker dropdown box
    # could select the ticker to view the Cumulative PnL of that Ticker (much cleaner!)
    container = widgets.HBox(children=[search, ticker])

    # Assign an emptry figure widget with two traces
    g = go.FigureWidget(layout=go.Layout(
                            width=1000, height=500,
                            title=dict(
                                text='Cumulative PnL Overview'
                            ),
                        ))
    # create the line chart
    g.add_scatter(name='Cumulative PnL')
        
    def response(change):
            temp_df = df[(df['Ticker'] == ticker.value)]
            with g.batch_update():
                # x-axis to display the Date and y-axis to display the Moving Average Volume
                g.data[0].x = temp_df['Date'].tolist()
                g.data[0].y = temp_df['Cumulative PnL/Ticker'].tolist()
                g.layout.xaxis.title = 'Date'
                g.layout.yaxis.title = 'Cumulative PnL'
    
    def chTicker(change):
        if search.value:
            ticker.options = sorted(df[df['Ticker'].str.contains(search.value)]['Ticker'].unique())
        else:
            ticker.options = sorted(list(df['Ticker'].unique()))
    # these are to update each choice in the box selected by user        
    ticker.observe(response, names='value')
    search.observe(chTicker, names='value')
                       
    response(None)
    chTicker(None)
    # to display the boxes
    display(container)
    display(g)

In [15]:
PnLBrowser(scaffold_table)

FigureWidget({
    'data': [{'name': 'Cumulative PnL',
              'type': 'scatter',
              'uid': '…

In [16]:
def PnLScatterBrowser(df):

    search = widgets.Text(value = '', placeholder='Search string', description='', disabled=False, continuous_update=True)
        
    ticker = widgets.Dropdown(
        options=sorted(list(df['Ticker'].unique())),
        value=sorted(df['Ticker'].unique())[0],
        description='Ticker: ',
    )
    
    # display the use_year ticker search box and ticker dropdown box
    # could select the ticker to view the Cumulative PnL of that Ticker (much cleaner!)
    container = widgets.HBox(children=[search, ticker])

    # Assign an emptry figure widget with two traces
    g = go.FigureWidget(layout=go.Layout(
                            width=1000, height=500,
                            title=dict(
                                text='Daily PnL (year1 vs year2)'
                            ),
                        ))
    # create the scatter plot, set the mode to markers
    g.add_scatter(name='Cumulative PnL', mode='markers')
    
    # these are the dates of the first half of each ticker's purchased duration
    # will be used to display the first half vs second half PnL scatter plot
    dict_first_half = {'AMZN': '2021-05-13', 'GOOG':'2021-05-13', 'AMD':'2021-06-15', 'AMCR':'2021-10-07', 
                  'AXP':'2021-11-09', 'MO':'2022-02-09', 'AEP':'2022-02-16', 'AAL':'2022-03-03'}
    
    def response(change):
            temp_df = df[(df['Ticker'] == ticker.value)]
            with g.batch_update():
                # x-axis to display the daily PnL of the first half and y-axis to display that of the second half
                g.data[0].x = temp_df[temp_df.Date < dict_first_half[ticker.value]]['Daily PnL'].tolist()
                g.data[0].y = temp_df[temp_df.Date >= dict_first_half[ticker.value]]['Daily PnL'].tolist()
                # create a title for x and y-axis
                g.layout.xaxis.title = 'First Half PnL'
                g.layout.yaxis.title = 'Second Half PnL'
                
    
    def chTicker(change):
        if search.value:
            ticker.options = sorted(df[df['Ticker'].str.contains(search.value)]['Ticker'].unique())
        else:
            ticker.options = sorted(list(df['Ticker'].unique()))
            
    # these are to update each choice in the box selected by user        
    ticker.observe(response, names='value')
    search.observe(chTicker, names='value')
                       
    response(None)
    chTicker(None)
    # to display the boxes
    display(container)
    display(g)

In [17]:
PnLScatterBrowser(scaffold_table)

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'Cumulative PnL',
              'type': …